In [1]:
output_dir = "/home/jovyan/20-restaurant-sales/data/out"

# Function to retrieve data from kaggle for the train component

In [2]:
def get_data(download_url, out_path):
    import subprocess
    import pickle
    import sys
    import subprocess
    
    
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    
    import pandas as pd
    
    
    default_url = "https://storage.googleapis.com/my-mldata/restaurant-revenue-prediction.zip"
    url = download_url if download_url else default_url
    
    subprocess.run(["wget", "-O", "restaurant-revenue-prediction.zip", url])
    
    subprocess.run(["unzip", "restaurant-revenue-prediction.zip"])
    subprocess.run(["unzip", "*.zip"])
    print("Files Unzipped....")
    subprocess.call(["rm", "-r", "*.zip"])
    
    train_data = pd.read_csv("train.csv")
    test_data = pd.read_csv("test.csv")
    
    with open(f"{out_path}/train_data.pkl", "wb") as train:
        pickle.dump(train_data, train)
        
    with open(f"{out_path}/test_data.pkl", "wb") as test:
        pickle.dump(test_data, test)
    

In [3]:
get_data("", output_dir)

Files Unzipped....


# Function to preprocess the data using correlation analysis

In [4]:
def preprocess_corr_analysis(out_path):
    
    import sys
    import subprocess
    
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
#     subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    
    import pickle
    import numpy as np
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.model_selection import train_test_split
    import pandas as pd
    
    with open(f"{out_path}/train_data.pkl", 'rb') as f:
        train = pickle.load(f)
        
    with open(f"{out_path}/test_data.pkl", 'rb') as f:
        test = pickle.load(f) 
       
    train.drop(['Id', 'Open Date', 'City'], axis=1, inplace=True)
    test.drop(['Id', 'Open Date', 'City'], axis=1, inplace=True)
    
    city_group_encoder = LabelEncoder()
    scaler = StandardScaler()

    train['City Group'] = city_group_encoder.fit_transform(train['City Group'])
    test['City Group'] = city_group_encoder.transform(test['City Group'])

    train = pd.get_dummies(train, drop_first=True)
    test = pd.get_dummies(test, drop_first=True)

    X = train.drop(['revenue'], axis=1)
    y = train[['revenue']]

    test.drop(["Type_MB"], axis=1, inplace=True)
    
    corr_matrix = X.corr().abs()
    
    condition = np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)
    upper = corr_matrix.where(condition)
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    
    X.drop(to_drop, axis=1, inplace=True)
    test.drop(to_drop, axis=1, inplace=True)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

    X_train = scaler.fit_transform(X_train)

    X_val = scaler.transform(X_val)

    test = scaler.transform(test)
    
#     outfile = TemporaryFile()
    np.savez_compressed(f'{out_path}/corr-analysis-data.npz',
                       X_train=X_train,
                       X_val=X_val,
                       y_train=y_train,
                       y_val=y_val,
                       test_data=test)
    print("Preprocessing Complete")

In [5]:
# preprocess_corr_analysis(output_dir)

In [6]:
def preprocess_pca_analysis(out_path):
    
    import sys
    import subprocess
    
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])

      
    import pickle
    import numpy as np
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.model_selection import train_test_split
    from sklearn.decomposition import PCA
    import pandas as pd
    
    with open(f"{out_path}/train_data.pkl", 'rb') as f:
        train = pickle.load(f)
        
    with open(f"{out_path}/test_data.pkl", 'rb') as f:
        test = pickle.load(f) 
       
    train.drop(['Id', 'Open Date', 'City'], axis=1, inplace=True)
    test.drop(['Id', 'Open Date', 'City'], axis=1, inplace=True)
    
    city_group_encoder = LabelEncoder()
    scaler = StandardScaler()

    train['City Group'] = city_group_encoder.fit_transform(train['City Group'])
    test['City Group'] = city_group_encoder.transform(test['City Group'])

    train = pd.get_dummies(train, drop_first=True)
    test = pd.get_dummies(test, drop_first=True)

    X = train.drop(['revenue'], axis=1)
    y = train[['revenue']]

    test.drop(["Type_MB"], axis=1, inplace=True)
    
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

    X_train = scaler.fit_transform(X_train)

    X_val = scaler.transform(X_val)

    test = scaler.transform(test)
    
    pca = PCA(n_components=2)
    
    X_train = pca.fit_transform(X_train)
    
    X_val = pca.transform(X_val)

    test = pca.transform(test)
    
    print(test[:3, :])
#     outfile = TemporaryFile()
    np.savez_compressed(f'{out_path}/pca-analysis-data.npz',
                       X_train=X_train,
                       X_val=X_val,
                       y_train=y_train,
                       y_val=y_val,
                       test_data=test)
    print("Preprocessing Complete")

# Function to train the data with Linear Regression Model with Ridge regularization - train component (train-ridge)

In [7]:
def train_corr_ridge(out_path, ridge_corr_file):
    
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    from sklearn.linear_model import Ridge
    from sklearn.metrics import mean_squared_error
    import numpy as np
    import pickle
    
    preprocessed_data = np.load(f'{out_path}/corr-analysis-data.npz')
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    
    X_val = preprocessed_data['X_val']
    y_val = preprocessed_data['y_val']
    ridge = Ridge(alpha=0.1)
    ridge.fit(X_train, y_train)
    
    y_pred = ridge.predict(X_val)
    
    with open(f'{out_path}/{ridge_corr_file}', "wb") as model:
        pickle.dump(ridge, model)
    
    RMSE = np.sqrt(mean_squared_error(y_pred, y_val))
    
    print(f"RMSE: {RMSE}")

    
    
    
    

In [8]:
# train_corr_ridge(output_dir, 'ridge-corr-model.pkl')

In [9]:
def train_pca_ridge(out_path, ridge_pca_file):
    
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    from sklearn.linear_model import Ridge
    from sklearn.metrics import mean_squared_error
    import numpy as np
    import pickle
    
    preprocessed_data = np.load(f'{out_path}/pca-analysis-data.npz')
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    
    X_val = preprocessed_data['X_val']
    y_val = preprocessed_data['y_val']
    ridge = Ridge(alpha=0.1)
    ridge.fit(X_train, y_train)
    
    y_pred = ridge.predict(X_val)
    
    with open(f'{out_path}/{ridge_pca_file}', "wb") as model:
        pickle.dump(ridge, model)
    
    RMSE = np.sqrt(mean_squared_error(y_pred, y_val))
    
    print(f"RMSE: {RMSE}")

    
    
    
    

# Function to train the data with Linear Regression Model with Lasso regularization - train component (train-lasso)

In [10]:
def train_corr_lasso(out_path, lasso_corr_file):
    
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    from sklearn.linear_model import Lasso
    from sklearn.metrics import mean_squared_error
    import numpy as np
    import pickle
    
    preprocessed_data = np.load(f'{out_path}/corr-analysis-data.npz')
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    
    X_val = preprocessed_data['X_val']
    y_val = preprocessed_data['y_val']
    lasso = Lasso(alpha=0.1)
    lasso.fit(X_train, y_train)
    
    y_pred = lasso.predict(X_val)
    
    with open(f'{out_path}/{lasso_corr_file}', "wb") as model:
        pickle.dump(lasso, model)
    
    RMSE = np.sqrt(mean_squared_error(y_pred, y_val))
    
    print(f"RMSE(Lasso) - Corr: {RMSE}")

    
    
    
    

In [11]:
def train_pca_lasso(out_path, lasso_pca_file):
    
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    from sklearn.linear_model import Lasso
    from sklearn.metrics import mean_squared_error
    import numpy as np
    import pickle
    
    preprocessed_data = np.load(f'{out_path}/pca-analysis-data.npz')
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    
    X_val = preprocessed_data['X_val']
    y_val = preprocessed_data['y_val']
    lasso = Lasso(alpha=0.1)
    lasso.fit(X_train, y_train)
    
    y_pred = lasso.predict(X_val)
    
    with open(f'{out_path}/{lasso_pca_file}', "wb") as model:
        pickle.dump(lasso, model)
    
    RMSE = np.sqrt(mean_squared_error(y_pred, y_val))
    print(X_train[:5, :])
    print(f"RMSE(Lasso|PCA): {RMSE}")

    
    
    
    

In [12]:
def train_corr_xgb(out_path, xgb_corr_file):
    
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn', 'xgboost'])
    from sklearn.metrics import mean_squared_error
    import numpy as np
    import pickle
    import xgboost as xgb
    
    preprocessed_data = np.load(f'{out_path}/corr-analysis-data.npz')
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    X_val = preprocessed_data['X_val']
    y_val = preprocessed_data['y_val']
    
    xg_reg = xgb.XGBRegressor(objective='reg:linear', colsample_bytree=0.3,
                              learning_rate=0.1, max_depth=5, alpha=10, n_estimators=10)

    xg_reg.fit(X_train, y=y_train)
    
    y_pred = xg_reg.predict(X_val)
    
    with open(f'{out_path}/{xgb_corr_file}', "wb") as model:
        pickle.dump(xg_reg, model)
    
    RMSE = np.sqrt(mean_squared_error(y_pred, y_val))
    
    print(f"RMSE(XGBoost): {RMSE}")

    
    
    
    

In [13]:
def train_pca_xgb(out_path, xgb_pca_file):
    
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn', 'xgboost'])
    from sklearn.metrics import mean_squared_error
    import numpy as np
    import pickle
    import xgboost as xgb
    
    preprocessed_data = np.load(f'{out_path}/pca-analysis-data.npz')
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    X_val = preprocessed_data['X_val']
    y_val = preprocessed_data['y_val']
    
    xg_reg = xgb.XGBRegressor(objective='reg:linear', colsample_bytree=0.3,
                              learning_rate=0.1, max_depth=5, alpha=10, n_estimators=10)

    xg_reg.fit(X_train, y=y_train)
    
    y_pred = xg_reg.predict(X_val)
    
    with open(f'{out_path}/{xgb_pca_file}', "wb") as model:
        pickle.dump(xg_reg, model)
    
    print(X_train[:3, :])
    RMSE = np.sqrt(mean_squared_error(y_pred, y_val))
    
    print(f"RMSE(XGBoost)| PCA: {RMSE}")

    
    
    
    

# Function to predict with Linear Regression Model with Ridge regularization - train component (predict-ridge)

In [14]:
def predict_corr_ridge(out_path, ridge_corr_file):
    
    import pickle
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    import numpy as np
    
    with open(f'{out_path}/{ridge_corr_file}', "rb") as model:
        ridge = pickle.load(model)
        
    test_data = np.load(f'{out_path}/corr-analysis-data.npz')['test_data']
    
    predictions = ridge.predict(test_data)
    
    np.savetxt(f'{out_path}/ridge-corr.txt', predictions, fmt='%1.2f')
    
#     sample_submission = pd.read_csv()
    print("File saved as ridge-corr.txt")

In [15]:
def predict_pca_ridge(out_path, ridge_pca_file):
    
    import pickle
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    import numpy as np
    
    with open(f'{out_path}/{ridge_pca_file}', "rb") as model:
        ridge = pickle.load(model)
        
    test_data = np.load(f'{out_path}/pca-analysis-data.npz')['test_data']
    
    predictions = ridge.predict(test_data)
    
    np.savetxt(f'{out_path}/ridge-pca.txt', predictions, fmt='%1.2f')
    
#     sample_submission = pd.read_csv()
    print("File saved as ridge-pca.txt")

In [16]:
def predict_corr_lasso(out_path, lasso_corr_file):
    
    import pickle
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    import numpy as np
    
    with open(f'{out_path}/{lasso_corr_file}', "rb") as model:
        lasso = pickle.load(model)
        
    test_data = np.load(f'{out_path}/corr-analysis-data.npz')['test_data']
    
    predictions = lasso.predict(test_data)
    
    np.savetxt(f'{out_path}/lasso-corr.txt', predictions, fmt='%1.2f')
    
#     sample_submission = pd.read_csv()
    print("File saved as lasso-corr.txt")

In [17]:
def predict_pca_lasso(out_path, lasso_pca_file):
    
    import pickle
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    import numpy as np
    
    with open(f'{out_path}/{lasso_pca_file}', "rb") as model:
        lasso = pickle.load(model)
        
    test_data = np.load(f'{out_path}/pca-analysis-data.npz')['test_data']
    print("==========================================================================================================\n")
    print(test_data[:3, :])
    print("==========================================================================================================\n")
    predictions = lasso.predict(test_data)
    
    np.savetxt(f'{out_path}/lasso-pca.txt', predictions, fmt='%1.2f')
    
#     sample_submission = pd.read_csv()
    print("File saved as lasso-pca.txt")

In [18]:
def predict_corr_xgb(out_path, xgb_corr_file):
    
    import pickle
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn', 'xgboost'])
    import numpy as np
    import xgboost
    
    with open(f'{out_path}/{xgb_corr_file}', "rb") as model:
        xgb = pickle.load(model)
        
    test_data = np.load(f'{out_path}/corr-analysis-data.npz')['test_data']
    
    predictions = xgb.predict(test_data)
    
    np.savetxt(f'{out_path}/xgb-corr.txt', predictions, fmt='%1.2f')
    
    print("File saved as xgb-corr.txt")

In [19]:
def predict_pca_xgb(out_path, xgb_pca_file):
    
    import pickle
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn', 'xgboost'])
    import numpy as np
    import xgboost
    
    with open(f'{out_path}/{xgb_pca_file}', "rb") as model:
        xgb = pickle.load(model)
        
    test_data = np.load(f'{out_path}/pca-analysis-data.npz')['test_data']
    print(test_data[:3, :])
    predictions = xgb.predict(test_data)
    
    np.savetxt(f'{out_path}/xgb-pca.txt', predictions, fmt='%1.2f')
    
    print("File saved as xgb-pca.txt")

In [20]:
# predict_ridge(output_dir, 'corr-ridge-model.pkl')

# Function to predict with Linear Regression Model with Ridge regularization - train component (predict-ridge)

In [21]:
# 

In [22]:
import kfp
from kfp import dsl
import kfp.components as comp

In [23]:
download_op = comp.func_to_container_op(get_data, base_image="python:3.7")

preprocess_corr_op = comp.func_to_container_op(preprocess_corr_analysis, base_image="tensorflow/tensorflow:latest-gpu-py3")
preprocess_pca_op = comp.func_to_container_op(preprocess_pca_analysis, base_image="tensorflow/tensorflow:latest-gpu-py3")

train_corr_ridge_op = comp.func_to_container_op(train_corr_ridge, base_image="tensorflow/tensorflow:latest-gpu-py3")
train_corr_lasso_op = comp.func_to_container_op(train_corr_lasso, base_image="tensorflow/tensorflow:latest-gpu-py3")
train_corr_xgb_op = comp.func_to_container_op(train_corr_xgb, base_image="tensorflow/tensorflow:latest-gpu-py3")

predict_corr_ridge_op = comp.func_to_container_op(predict_corr_ridge, base_image="tensorflow/tensorflow:latest-gpu-py3")
predict_corr_lasso_op = comp.func_to_container_op(predict_corr_lasso, base_image="tensorflow/tensorflow:latest-gpu-py3")
predict_corr_xgb_op = comp.func_to_container_op(predict_corr_xgb, base_image="tensorflow/tensorflow:latest-gpu-py3")

train_pca_ridge_op = comp.func_to_container_op(train_pca_ridge, base_image="tensorflow/tensorflow:latest-gpu-py3")
train_pca_lasso_op = comp.func_to_container_op(train_pca_lasso, base_image="tensorflow/tensorflow:latest-gpu-py3")
train_pca_xgb_op = comp.func_to_container_op(train_pca_xgb, base_image="tensorflow/tensorflow:latest-gpu-py3")

predict_pca_ridge_op = comp.func_to_container_op(predict_pca_ridge, base_image="tensorflow/tensorflow:latest-gpu-py3")
predict_pca_lasso_op = comp.func_to_container_op(predict_pca_lasso, base_image="tensorflow/tensorflow:latest-gpu-py3")
predict_pca_xgb_op = comp.func_to_container_op(predict_pca_xgb, base_image="tensorflow/tensorflow:latest-gpu-py3")

In [24]:
@dsl.pipeline(
    name="Restaurant Sales Revenue Prediction",
    description="A Machine Learning Pipeline for Revenue prediction model"
)

def revenue_prediction_pipeline(
    out_path: str,
    ridge_corr_file: str,
    lasso_corr_file: str,
    xgb_corr_file: str,
    ridge_pca_file: str,
    lasso_pca_file: str,
    xgb_pca_file: str,
    download_url: str
):
    volume_op = dsl.VolumeOp(
        name="create_volume",
        resource_name="data-volume",
        size="1Gi",
        modes=dsl.VOLUME_MODE_RWO)
    
    download_container = download_op(download_url, out_path).add_pvolumes({out_path: volume_op.volume})
    
    preprocess_corr_container = preprocess_corr_op(out_path).add_pvolumes({out_path: download_container.pvolume})
    
    train_ridge_corr_container = train_corr_ridge_op(out_path, ridge_corr_file) \
                                        .add_pvolumes({out_path: preprocess_corr_container.pvolume})
    
    train_lasso_corr_container = train_corr_lasso_op(out_path, lasso_corr_file) \
                                        .add_pvolumes({out_path: preprocess_corr_container.pvolume})
    
    train_xgb_corr_container = train_corr_xgb_op(out_path, xgb_corr_file) \
                                        .add_pvolumes({out_path: preprocess_corr_container.pvolume})
    
    predict_ridge_corr_container = predict_corr_ridge_op(out_path, ridge_corr_file) \
                                        .add_pvolumes({out_path: train_ridge_corr_container.pvolume})
    
    predict_lasso_corr_container = predict_corr_lasso_op(out_path, lasso_corr_file) \
                                        .add_pvolumes({out_path: train_lasso_corr_container.pvolume})
    
    predict_xgb_corr_container = predict_corr_xgb_op(out_path, xgb_corr_file) \
                                        .add_pvolumes({out_path: train_xgb_corr_container.pvolume})
    
    
    #PCA
    preprocess_pca_container = preprocess_pca_op(out_path).add_pvolumes({out_path: download_container.pvolume})
    
    train_ridge_pca_container = train_pca_ridge_op(out_path, ridge_pca_file) \
                                        .add_pvolumes({out_path: preprocess_pca_container.pvolume})
    
    train_lasso_pca_container = train_pca_lasso_op(out_path, lasso_pca_file) \
                                        .add_pvolumes({out_path: preprocess_pca_container.pvolume})
    
    train_xgb_pca_container = train_pca_xgb_op(out_path, xgb_pca_file) \
                                        .add_pvolumes({out_path: preprocess_pca_container.pvolume})
    
    predict_ridge_pca_container = predict_pca_ridge_op(out_path, ridge_pca_file) \
                                        .add_pvolumes({out_path: train_ridge_pca_container.pvolume})
    
    predict_lasso_pca_container = predict_pca_lasso_op(out_path, lasso_pca_file) \
                                        .add_pvolumes({out_path: train_lasso_pca_container.pvolume})
    
    predict_xgb_pca_container = predict_pca_xgb_op(out_path, xgb_pca_file) \
                                        .add_pvolumes({out_path: train_xgb_pca_container.pvolume})
    
    
    ridge_corr_result_container = dsl.ContainerOp(
        name="predictions",
        image="library/bash:4.4.23",
        pvolumes={out_path: predict_ridge_corr_container.pvolume},
        arguments=['head', f'{out_path}/ridge-corr.txt']
    )
    
    
    lasso_corr_result_container = dsl.ContainerOp(
        name="predictions",
        image="library/bash:4.4.23",
        pvolumes={out_path: predict_lasso_corr_container.pvolume},
        arguments=['head', f'{out_path}/lasso-corr.txt']
    )
    
    
    xgb_corr_result_container = dsl.ContainerOp(
        name="predictions",
        image="library/bash:4.4.23",
        pvolumes={out_path: predict_xgb_corr_container.pvolume},
        arguments=['head', f'{out_path}/xgb-corr.txt']
    )
    
    ridge_pca_result_container = dsl.ContainerOp(
        name="predictions",
        image="library/bash:4.4.23",
        pvolumes={out_path: predict_ridge_pca_container.pvolume},
        arguments=['head', f'{out_path}/ridge-pca.txt']
    )
    
    
    lasso_pca_result_container = dsl.ContainerOp(
        name="predictions",
        image="library/bash:4.4.23",
        pvolumes={out_path: predict_lasso_pca_container.pvolume},
        arguments=['head', f'{out_path}/lasso-pca.txt']
    )
    
    
    xgb_pca_result_container = dsl.ContainerOp(
        name="predictions",
        image="library/bash:4.4.23",
        pvolumes={out_path: predict_xgb_pca_container.pvolume},
        arguments=['head', f'{out_path}/xgb-pca.txt']
    )

In [25]:
OUT_PATH = '/mnt'

DOWNLOAD_URL = ""

RIDGE_CORR_MODEL_PATH = 'ridge-corr-model.pkl'
LASSO_CORR_MODEL_PATH = 'lasso-corr-model.pkl'
XGB_CORR_MODEL_PATH = 'xgb-corr-model.pkl'

RIDGE_PCA_MODEL_PATH = 'ridge-pca-model.pkl'
LASSO_PCA_MODEL_PATH = 'lasso-pca-model.pkl'
XGB_PCA_MODEL_PATH = 'xgb-pca-model.pkl'



In [26]:
pipeline_func = revenue_prediction_pipeline

In [27]:
experiment_name = 'kubeflow_revenue_prediction'
run_name = f'{pipeline_func.__name__} run'
arguments = {
    "out_path": OUT_PATH,
    "ridge_corr_file": RIDGE_CORR_MODEL_PATH,
    "lasso_corr_file": LASSO_CORR_MODEL_PATH,
    "xgb_corr_file": XGB_CORR_MODEL_PATH,
    "ridge_pca_file": RIDGE_PCA_MODEL_PATH,
    "lasso_pca_file": LASSO_PCA_MODEL_PATH,
    "xgb_pca_file": XGB_PCA_MODEL_PATH,
    "download_url": DOWNLOAD_URL
}

kfp.compiler.Compiler().compile(pipeline_func, f'{experiment_name}.zip')



In [28]:
client = kfp.Client()

In [29]:
run_result = client.create_run_from_pipeline_func(pipeline_func,
                                                 experiment_name=experiment_name,
                                                 run_name=run_name,
                                                 arguments=arguments
                                                 )